# 추천시스템 알고리즘 쇼케이스

## 1. module loading

In [1]:
import numpy as np
import pandas as pd

In [2]:
def softmax(a) :
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y=exp_a / sum_exp_a
    
    return y

In [18]:
def weighted_initial_user_weight(company_list,company_df):
    '''
    Input
    
    company_list:사용자가 회원가입 과정에서 선택하게 된 기업 리스트 (python의 리스트로 input)
    
    '''
    if len(company_list)==0:
        user_weight=1/9*np.ones((1,9)).reshape([1,9])
    else:
        user_weight=company_df[company_df['회사명'].isin(company_list)].mean(axis=0).to_numpy().reshape([1,9])
    
    #선호 기업으로부터 얻은 주제 분포에 정보량의 역수만큼 weight 부여
    weighted_user_weight=user_weight/-np.log(user_weight) 
    
    initial_user_weight=weighted_user_weight/weighted_user_weight.sum()
    
    
    return initial_user_weight

In [19]:
def attention_recommendation(company_list,company_df,news_df):
    '''
    input:
    company_list: register에서 회원가입할때 post로 넘어오는 데이터 중 선호 기업에 대한 리스트
    company_df: 기업별 주제 분포, 폴더 내 company_matrix.csv를 그대로 사용하면 됨
    news_df: 뉴스별 주제 분포, 폴더 내 recommendation_matrix.csv를 그대로 사용하면 됨
    
    Output:
    news_id_list.tolist() : 추천 결과에 맞는 뉴스의 id값을 리스트 형태로 반환
    
    자세한 사용 내역은 밑에 3. Test 참고!
    '''
    
    
    #데이터프레임으로 받은 기사별 주제가중치를 numpy의 array형으로 바꾸기
    target=news_df['_id']
    news_df_array=news_df.drop(['_id'],axis=1).to_numpy()
    
    #사용자의 initial 주제 가중치 구하기
    user_dist=weighted_initial_user_weight(company_list,company_df)
    
    #Attention 개념을 응용한 추천 시스템 구현 파트
    #사용자 주제 가중치와 뉴스 가중치를 내적하여 Attention Score 구하기
    at_score=user_dist.dot(news_df_array.T)
    
    #Attention Score에 Softmax를 적용하여 각 기사에 얼마나 '집중'할지 계산
    at_dist=softmax(at_score)
    
    #Attention distribution에서 가장 높은 값 3개를 찾아 인덱스 반환
    arg_list=np.argsort(at_dist,axis=1)[:,93:96]
    
    #해당 인덱스에 맞는 뉴스 추천
    news_id_list=target.iloc[arg_list.tolist()[0]]
    
    return news_id_list.tolist()

## 2. Dataset

In [20]:
company=pd.read_csv('company_matrix.csv',encoding='utf-8')
news=pd.read_csv('recommendation_matrix.csv',encoding='utf-8')

## 3. Test


In [21]:
import time

In [26]:
check_news=pd.read_csv('데이터 확인용_사용x.csv',encoding='utf-8')

1) 선호기업: 카카오, SK, 케이티

In [55]:
start = time.time()
result1=attention_recommendation(['카카오','SK','케이티'],company,news)
end = time.time()

print(f"{end - start:.5f} sec")

0.05687 sec


C:\Users\wpekt\AppData\Local\Temp/ipykernel_28916/3351391866.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  user_weight=company_df[company_df['회사명'].isin(company_list)].mean(axis=0).to_numpy().reshape([1,9])


In [56]:
check_news[check_news['_id'].isin(result1)]

,_id,Date,article_name,reporter,image,article_main,article_summary,keywords,__v,X1,X2,X3,X4,X5,X6,X7,X8,X9
0,6371d62d56e914c0d68a6995,2022-10-16T00:00:00.000Z,12시간 먹통된 카톡 다시 수면위로 오른 재난관리기본계획 개정,김만기,https://image.fnnews.com/resource/media/image/...,SK㈜ C&C의 경기도 판교 데이터센터 화재로 메신저 카카오톡 등이 12시간 넘...,SK(주) C&C의 경기도 판교 데이터센터 화재로 메신저 카카오톡 등이 12시간 넘...,"['데이터센터', '화재', 'SK']",0,0.460946,0.064906,0.055006,0.064906,0.055006,0.055006,0.104510,0.074807,0.064906
25,6371ec9b56e914c0d68a69d3,2022-10-15T00:00:00.000Z,판교 데이터센터 화재로 카카오 서비스 무더기 장애,김대기,https://file.mk.co.kr/meet/neds/2022/10/image_...,SK C&C ''화재 전원 차단으로 서비스 장애데이터센터 정상화 총력''카카...,15일 카카오가 이용하고 있는 판교 SK C&C 데이터센터에서 화재가 발생해 주요 ...,"['화재', '접속', '이날', '있다.', '서비스', '오후', '데이터센터'...",0,0.500573,0.057274,0.057274,0.057274,0.067583,0.067583,0.067583,0.067583,0.057274
64,63720e1f56e914c0d68a6a28,2022-10-23T00:00:00.000Z,"카카오, 사상초유 '먹통'에 공매도 타깃됐다",이민지,https://cphoto.asiae.co.kr/listimglink/1/20221...,1.4兆 바이코리아…5주연속 순매수 랠리 외인들의 코스피 편애…코스닥은 팔았다코스피...,지난 일주일(17∼21일) 동안 공매도 물량을 보면 카카오 035720 | 코스피 ...,"['15:30', '장마감', '현재가', '카카오', '등락률', '코스피', '...",0,0.555556,0.047081,0.064030,0.064030,0.047081,0.080979,0.047081,0.047081,0.047081


2) 선호기업: 삼성바이오로직스, 셀트리온, 대웅제약

In [63]:
start = time.time()
result2=attention_recommendation(['삼성바이오로직스', '셀트리온', '대웅제약'],company,news)
end = time.time()

print(f"{end - start:.5f} sec")

0.06084 sec


C:\Users\wpekt\AppData\Local\Temp/ipykernel_28916/3351391866.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  user_weight=company_df[company_df['회사명'].isin(company_list)].mean(axis=0).to_numpy().reshape([1,9])


In [64]:
check_news[check_news['_id'].isin(result2)]

,_id,Date,article_name,reporter,image,article_main,article_summary,keywords,__v,X1,X2,X3,X4,X5,X6,X7,X8,X9
39,6371f31d56e914c0d68a69f4,2022-09-28T00:00:00.000Z,"韓, 바이든 IRA로 장기적으로는 다른 국가보다 이득",윤재준,https://image.fnnews.com/resource/media/image/...,미국 조 바이든 행정부가 북미 밖에서 생산된 전기차를 보조금 혜택 대상에서 제...,미국 조 바이든 행정부가 북미 밖에서 생산된 전기차를 보조금 혜택 대상에서 제외하면...,"['한국', '배터리', '미국', '업체들', '공장', '생산', '중국', '...",0,0.056809,0.041771,0.049290,0.425230,0.079365,0.049290,0.162072,0.064327,0.071846
81,6372344856e914c0d68a6a50,2022-10-11T00:00:00.000Z,"식약처, 삼바 제조 모더나 코로나19 '2가백신' 국가출하승인",강중모,https://image.fnnews.com/resource/media/image/...,관련종목▶삼성바이오로직스(207940) 식품의약품안전처는 모더나코리아의 국내 제...,식품의약품안전처는 지난달 8일 허가받은 ‘모더나스파이크박스2주’와 같은 원료의약품을...,['백신'],0,0.070613,0.051921,0.107996,0.491173,0.051921,0.051921,0.070613,0.051921,0.051921
85,6372369f56e914c0d68a6a58,2022-10-11T00:00:00.000Z,"진격의 거인'된 삼바, 압도적 생산력으로 CMO 패권 잡는다",이춘희,https://cphoto.asiae.co.kr/listimglink/1/20221...,삼성바이오로직스 삼성바이오로직스 207940 | 코스피 증권정보 ...,삼성바이오로직스가 세계 바이오 위탁개발생산 톱 티어를 향한 발판이 될 4공장을 가동...,"['생산', '삼성바이오로직스', '능력', '확보', '공장', '있다', '바이...",0,0.062422,0.084894,0.062422,0.433208,0.096130,0.062422,0.073658,0.062422,0.062422


3) 선호기업: 삼성전자, LG에너지솔루션, SK하이닉스

In [61]:
start = time.time()
result3=attention_recommendation(['삼성전자','LG에너지솔루션','SK하이닉스'],company,news)
end = time.time()

print(f"{end - start:.5f} sec")

0.09575 sec


C:\Users\wpekt\AppData\Local\Temp/ipykernel_28916/3351391866.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  user_weight=company_df[company_df['회사명'].isin(company_list)].mean(axis=0).to_numpy().reshape([1,9])


In [62]:
check_news[check_news['_id'].isin(result3)]

,_id,Date,article_name,reporter,image,article_main,article_summary,keywords,__v,X1,X2,X3,X4,X5,X6,X7,X8,X9
40,6371f3cf56e914c0d68a69f6,2022-10-17T00:00:00.000Z,"SK이노베이션, 3분기 영업익 컨센서스 하회 전망 배터리는 적자 줄 듯-한투",홍재영,https://thumb.mt.co.kr/06/2022/10/202210170850...,"한국투자증권이 17일 SK이노베이션 (177,500원 ▼12,000 -6.33...",한국투자증권이 17일 SK이노베이션에 대해 정제마진 시황이 경기침체 우려에 민감하게...,"[""것''이라"", '전망']",0,0.083028,0.061050,0.061050,0.083028,0.061050,0.434676,0.061050,0.061050,0.094017
79,637219b856e914c0d68a6a4c,2022-10-06T00:00:00.000Z,"'KT, 통신 부문 성장 안정적 역사적 저평가 상태'-이베스트",신현아,https://img.hankyung.com/photo/202210/01.28563...,이베스트투자증권은 6일 KT에 대해 통신 부문의 안정적인 성장과 함께 자회자 실적 ...,이베스트투자증권은 6일 통신 부문의 안정적인 성장과 함께 자회자 실적 호조가 예상된...,"['증가', '예상']",0,0.054466,0.093682,0.103486,0.054466,0.054466,0.466231,0.054466,0.064270,0.054466
86,6372373356e914c0d68a6a5a,2022-10-17T00:00:00.000Z,"배당귀족ETF, 실적도 꽉 잡았다",김정범,https://img.hankyung.com/photo/202101/AD.25063...,절반이 영업이익 두자릿수 쑥8곳은 올해 50%이상 증가예상편입비중 높은 SK·GS...,17일 에프앤가이드에 따르면 TIGER MKF 배당귀족 상장지수펀드(ETF) 편입 ...,"['배당귀족', '전년', '올해', '것으로', '예상', 'S&P500', '영...",0,0.072650,0.053419,0.053419,0.072650,0.053419,0.505342,0.053419,0.082265,0.053419


4) 선호기업: 한국전력공사, 한국가스공사

In [67]:
start = time.time()
result4=attention_recommendation(['한국전력공사','한국가스공사'],company,news)
end = time.time()

print(f"{end - start:.5f} sec")

0.05087 sec


C:\Users\wpekt\AppData\Local\Temp/ipykernel_28916/3351391866.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  user_weight=company_df[company_df['회사명'].isin(company_list)].mean(axis=0).to_numpy().reshape([1,9])


In [68]:
check_news[check_news['_id'].isin(result4)]

,_id,Date,article_name,reporter,image,article_main,article_summary,keywords,__v,X1,X2,X3,X4,X5,X6,X7,X8,X9
13,6371e70256e914c0d68a69b7,2022-10-27T00:00:00.000Z,"KT LG유플러스, 문체부 상대 'OTT 음악저작권료 소송' 1심서 패소",이정화,https://image.fnnews.com/resource/media/image/...,KT와 LG유플러스가 온라인 동영상 서비스(OTT) 음악 저작권료 인상안에 대응해 ...,KT와 LG유플러스가 온라인 동영상 서비스(OTT) 음악 저작권료 인상안에 대응해 ...,['OTT'],0,0.056385,0.093698,0.048922,0.086235,0.041459,0.056385,0.504146,0.041459,0.071310
32,6371eed156e914c0d68a69e1,2022-09-30T00:00:00.000Z,미혼부 출생신고' 쉬워진다 규제심판부 첫 ''개선 권고'',강진규,https://img.hankyung.com/photo/202209/02.25002...,대통령 직속 규제개혁위원회 산하 규제심판부는 30일 미혼부 아동 출생신고 제도개선...,대통령 직속 규제개혁위원회 산하 규제심판부는 30일 미혼부가 출생신고 과정에서 겪는...,"['출생신고', '미혼부', '규제', '아이', '개선']",0,0.073355,0.053937,0.073355,0.063646,0.063646,0.053937,0.471413,0.073355,0.073355
69,637213c356e914c0d68a6a32,2022-10-20T00:00:00.000Z,"진경준 전 검사장, 징계부가금 취소 소송 항소심도 패소",성시호,https://thumb.mt.co.kr/06/2022/10/202210201519...,넥슨 공짜주식 사건에 연루돼 현직 검사장 최초로 해임된 진경준씨가 법무부를 상대...,서울고법 행정9-2부(부장판사 김승주·조찬영·강문경)는 20일 넥슨 공짜주식 사건에...,"['진씨', '징계부가금', '검사장', '법무부']",0,0.052444,0.044444,0.044444,0.060444,0.060444,0.068444,0.564444,0.044444,0.060444


5) 이미트, 오뚜기, 롯데제과, 롯데쇼핑

In [73]:
start = time.time()
result5=attention_recommendation(['이마트','오뚜기','롯데제과','롯데쇼핑'],company,news)
end = time.time()

print(f"{end - start:.5f} sec")

0.05681 sec


C:\Users\wpekt\AppData\Local\Temp/ipykernel_28916/3351391866.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  user_weight=company_df[company_df['회사명'].isin(company_list)].mean(axis=0).to_numpy().reshape([1,9])


In [74]:
check_news[check_news['_id'].isin(result5)]

,_id,Date,article_name,reporter,image,article_main,article_summary,keywords,__v,X1,X2,X3,X4,X5,X6,X7,X8,X9
11,6371e5f356e914c0d68a69b1,2022-09-29T00:00:00.000Z,"LG전자, 상권 활성화 위한 신개념 가전체험공간 선봬",박선미,https://cphoto.asiae.co.kr/listimglink/1/20220...,LG전자가 차별화된 고객 경험을 선사하는 신개념 가전체험공간 ‘어나더바이브’를 열고...,LG전자가 차별화된 고객 경험을 선사하는 신개념 가전체험공간 ‘어나더바이브’를 열고...,"['어나더바이브', 'LG']",0,0.053208,0.123631,0.489828,0.046166,0.081377,0.046166,0.060250,0.039124,0.060250
67,637211cd56e914c0d68a6a2e,2022-10-19T00:00:00.000Z,"삼성전자, 비스포크 슈드레서 한정판 출시 케이스스터디와 맞손",오진영,https://thumb.mt.co.kr/06/2022/10/202210190857...,삼성전자가 스니커즈 콘셉트 스토어 케이스스터디와 함께 비스포크 슈드레서 한정판을...,삼성전자가 M니커즈 콘셉트 스토어 케이스스터디와 함께 신발관리기 본연의 역할뿐 아니...,"['비스포크', '슈드레서', '케이스스터디']",0,0.057274,0.057274,0.479954,0.057274,0.098511,0.057274,0.077892,0.057274,0.057274
68,637212e256e914c0d68a6a30,2022-10-18T00:00:00.000Z,환절기 따뜻한 바람 솔솔 에어컨 공기청정기 변신,정서린,https://img.seoul.co.kr/img/upload/2022/10/17/...,최근 기온이 급격하게 떨어지고 큰 일교차로 아침저녁 쌀쌀함이 부쩍 체감되면서 온...,최근 기온이 급격하게 떨어지고 큰 일교차로 아침저녁 쌀쌀함이 부쩍 체감되면서 온풍을...,"['공기', '바람', '온도', '있다']",0,0.049164,0.093412,0.465093,0.049164,0.093412,0.066863,0.058014,0.066863,0.058014
